In [33]:
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urlparse
import file

indexUrl = 'https://read.kimetsu.online/'
chapterUrlPrefix = indexUrl + 'manga/kimetsu-no-yaiba-chapter-'

In [9]:
import urllib.request as urlreq
def get_page(url):
    req = urlreq.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36')
    content = urlreq.urlopen(req).read()
    return content

In [20]:
soup = BeautifulSoup(get_page(indexUrl))
chapters = [link['href'] for link in soup.find_all('a', href=True) if link['href'].startswith(chapterUrlPrefix)]
chapters

In [25]:
chUrl = chapters[0]
def get_images(chUrl):
    soup = BeautifulSoup(get_page(chUrl))
    images = [img['src'] for img in soup.find_all('img', src=True) if 'Kimetsu' in img['alt']]
    return images

In [26]:
images = dict()
for chUrl in chapters:
    cName = chUrl.replace(chapterUrlPrefix, '').replace('/','')
    images[cName] = get_images(chUrl)
images

{'72': ['https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/1.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/2.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/3.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/4.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/5.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/6.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/7.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/8.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/9.jpg',
  'https://cdn.hxmanga.com/file/majekayoo/kimetsu-no-yaiba/chapter-72-search-for-the-wives/10.jpg',
  '

In [43]:
def get_image_contents(cName, urls):
    try:
        os.mkdir(cName)
    except FileExistsError:
        pass
    for img in urls:
        fn = os.path.basename(urlparse(img).path)
        print(os.path.join(cName, fn))
        req = urlreq.Request(img)
        req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36')
        response = urlreq.urlopen(req)
        print(response.status)
        with open(os.path.join(cName, fn), 'wb') as f:
            f.write(response.read())

for cName, urls in images.items():
    get_image_contents(cName, urls)

72\1.jpg
200
72\2.jpg
200
72\3.jpg
200
72\4.jpg
200
72\5.jpg
200
72\6.jpg
200
72\7.jpg
200
72\8.jpg
200
72\9.jpg
200
72\10.jpg
200
72\11.jpg
200
72\12.jpg
200
72\13.jpg
200
72\14.jpg
200
72\15.jpg
200
72\16.jpg
200
72\17.jpg
200
72\18.jpg
200
72\19.jpg
200
205-5\1.jpg
200
205-5\2.jpg
200
205-5\3.jpg
200
205-5\4.jpg
200
205-5\5.jpg
200
205-5\6.jpg
200
205-5\7.jpg
200
205-5\8.jpg
200
205-5\9.jpg
200
205-5\10.jpg
200
205-5\11.jpg
200
205-5\12.jpg
200
205-5\13.jpg
200
205-5\14.jpg
200
205-5\15.jpg
200
205-5\16.jpg
200
205-5\17.jpg
200
205-5\18.jpg
200
205-5\19.jpg
200
205\1.jpg
200
205\2.jpg
200
205\3.jpg
200
205\4.jpg
200
205\5.jpg
200
205\6.jpg
200
205\7.jpg
200
205\8.jpg
200
205\9.jpg
200
205\10.jpg
200
205\11.jpg
200
205\12.jpg
200
205\13.jpg
200
205\14.jpg
200
205\15.jpg
200
205\16.jpg
200
205\17.jpg
200
205\18.jpg
200
205\19.jpg
200
205\20.jpg
200
205\21.jpg
200
205\22.jpg
200
205\23.jpg
200
205\24.jpg
200
204\1.jpg
200
204\2.jpg
200
204\3.jpg
200
204\4.jpg
200
204\5.jpg
200
204\6.jp

In [79]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from PIL import Image

def scale(imw, imh):
    page = A4
    a = imw/imh
    pa = page[0]/page[1]
    if pa < a:
        w = page[0]
        s = page[0] / imw
        h = imh * s
        x = 0
        y = (page[1] - h)/2
    else:
        h = page[1]
        s = page[1] / imh
        w = imw * s
        y = 0
        x = (page[0]-w) / 2
    return (x, y, w, h)

def create_pdf(cName, urls):
    can = canvas.Canvas(cName+'.pdf', A4)
    for img in urls:
        fn = os.path.join(cName, os.path.basename(urlparse(img).path))
        fi = Image.open(fn)
        x, y, w, h = scale(fi.width, fi.height)
        can.drawImage(fn, x, y, w, h)
        can.showPage()
    can.save()

for cName, urls in images.items():
    create_pdf(cName, urls)